In [1]:
from pathlib import Path
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import soundfile as sf
import pandas as pd

import os 
import itertools
import re
import math
import csv
import random
from sklearn import tree
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from scipy.fftpack import fftn, ifftn, fft, ifft
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy import signal
import pickle
from sklearn.externals import joblib
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
import warnings

warnings.filterwarnings('ignore') 
df = pd.read_csv("noAudible/Features_15too20hz.csv")



#df = pd.read_csv("Features_all_no_audible.csv")
#df = df[(df.modeID != 'audible')]
df

C:\Users\Magomed\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


,channel_1_mav,channel_1_mav2,channel_1_rms,channel_1_iav,channel_1_ssi,channel_1_var,channel_1_wl,channel_1_iemg,channel_1_aac,channel_1_zc,...,channel_6_aac,channel_6_zc,channel_6_ssc,channel_6_wamp,channel_6_medf,channel_6_meanf,userID,sessionID,uttID,modeID
0,0.003868,0.003480,0.005445,15.113432,0.115852,0.000030,2.772366,15.113432,0.000710,226,...,0.444535,1278,2241,1068,0.000000,66.053648,1,1001,10010001,audible
1,0.002984,0.002617,0.004356,7.843663,0.049889,0.000019,1.471776,7.843663,0.000560,155,...,0.443308,794,1497,728,0.000000,57.514921,1,1001,10010002,audible
2,0.005154,0.005004,0.007021,24.393531,0.233315,0.000049,4.531346,24.393531,0.000957,277,...,0.445901,1534,2726,1316,0.000000,73.430718,1,1001,10010003,audible
3,0.004691,0.004569,0.006249,22.794771,0.189746,0.000039,4.239613,22.794771,0.000873,290,...,0.445519,1540,2792,1345,1.859504,77.039314,1,1001,10010004,audible
4,0.004874,0.004688,0.006750,26.726812,0.249847,0.000046,4.799299,26.726812,0.000875,311,...,0.446329,1716,3143,1523,7.678245,77.906715,1,1001,10010005,audible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,0.003638,0.003164,0.005736,9.912880,0.089646,0.000033,1.740830,9.912880,0.000639,153,...,0.375292,1702,2214,389,0.000000,59.203804,8,8019,80190246,whisper
4796,0.002991,0.002215,0.005149,6.646271,0.058918,0.000027,1.203647,6.646271,0.000542,128,...,0.374308,1374,1823,317,0.000000,52.306202,8,8019,80190247,whisper
4797,0.003220,0.002680,0.004729,8.465624,0.058783,0.000022,1.531708,8.465624,0.000583,155,...,0.373218,1628,2147,375,0.000000,59.650508,8,8019,80190248,whisper
4798,0.002688,0.002088,0.004853,5.368740,0.047028,0.000024,0.964851,5.368740,0.000483,117,...,0.374330,1232,1644,285,0.000000,46.464064,8,8019,80190249,whisper


In [2]:
def split_data_per_session(session_id):
    temp_df = df[(df.sessionID == session_id)]
    data = temp_df.copy()
    data = shuffle(data)
    y = data['modeID']
    data.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return train_test_split(data, y, test_size=0.2)

def split_data_session(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['sessionID'].isin(train_user_ids)]
    test_df = data.loc[data['sessionID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

def split_data_utterance(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['userID'].isin(train_user_ids)]
    test_df = data.loc[data['userID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

def split_data(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['userID'].isin(train_user_ids)]
    test_df = data.loc[data['userID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

In [3]:
def lda_user1():
    train_ids = [2,3,4,5,6,7,8]
    test_ids = [1]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score     
def lda_user2():
    train_ids = [1,3,4,5,6,7,8]
    test_ids = [2]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score  
    
def lda_user3():
    train_ids = [1,2,4,5,6,7,8]
    test_ids = [3]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score 
    
def lda_user4():
    train_ids = [1,2,3,5,6,7,8]
    test_ids = [4]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score     
        
def lda_user5():
    train_ids = [1,2,3,4,6,7,8]
    test_ids = [5]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score   
    
def lda_user6():
    train_ids = [1,2,3,4,5,7,8]
    test_ids = [6]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score  
    
def lda_user7():
    train_ids = [1,2,3,4,5,6,8]
    test_ids = [7]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score    
    
def lda_user8():
    train_ids = [1,2,3,4,5,6,7]
    test_ids = [8]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score 

In [4]:
def lda_users():
    a = lda_user1()
    b = lda_user2()
    c = lda_user3()
    d = lda_user4()
    e = lda_user5()
    f = lda_user6()
    g = lda_user7()
    h = lda_user8()
    scores = np.array([a,b,c,d,e,f,g,h])
    print("Durchschnitt: ")
    print (scores.mean())
    print("Standartabweichung: ")
    print (scores.std())

In [5]:
lda_users()

Accuracy for session 19 is 0.37555555555555553
Accuracy for session 19 is 0.3253333333333333
Accuracy for session 19 is 0.34
Accuracy for session 19 is 0.43666666666666665
Accuracy for session 19 is 0.41333333333333333
Accuracy for session 19 is 0.43333333333333335
Accuracy for session 19 is 0.37666666666666665
Accuracy for session 19 is 0.40476190476190477
Durchschnitt: 
0.3882063492063492
Standartabweichung: 
0.038503853537190325
